In [1]:
import pandas
import os
from gene_panel_selection import ExpressionDataset, PROPOSEMethod, SeuratMethod

In [2]:
import torch.cuda
print("cuda devices:")
for i in range(torch.cuda.device_count()):
    print(f"  {i}: {torch.cuda.get_device_name(i)}")    

cuda devices:
  0: Quadro P1000


In [8]:
data_path = '../../../data/macaque_bg/'
expression = pandas.read_feather(data_path + 'counts.feather')
annotation = pandas.read_feather(data_path + 'anno.feather').set_index('sample_id')[['class_label', 'subclass_label', 'supertype_label']]

In [9]:
expression

,U6,ZNF692,ZNF672,SH3BP5L,OR2T27,Y-RNA,OR2T10,OR14I1,OR9G1,OR2T1,...,Metazoa-SRP ENSMMUG00000053951,U6 ENSMMUG00000064065,U3 ENSMMUG00000064655,U3 ENSMMUG00000061317,U3 ENSMMUG00000035418,Metazoa-SRP ENSMMUG00000053315,U4 ENSMMUG00000064166,Metazoa-SRP ENSMMUG00000054492,Metazoa-SRP ENSMMUG00000056295,Y-RNA ENSMMUG00000050721
sample_id,,,,,,,,,,,,,,,,,,,,,
AAACAGCCAAACATAG-L8XR_211104_02_D11-1141371085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAAACTGTT-L8XR_210812_01_H10-1124987481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAAAGGCCA-L8XR_211104_02_F11-1141371089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAAATTGCT-L8XR_210909_02_B09-1132816394,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAACAGCCAACCCTCC-L8XR_211104_02_A11-1141371084,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTGTTCAC-L8XR_210916_02_E10-1131607944,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTTTAACCC-L8XR_210909_02_C09-1132816395,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTTGTTGGTTTGGCGG-L8XR_211104_02_F11-1141371089,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
annotation

,class_label,subclass_label,supertype_label
sample_id,,,
AAACAGCCAGCTTAAT-L8XR_210812_01_A11-1124987480,MSN,D2-MSN,D2-Matrix
AAACAGCCAGGCGATA-L8XR_210812_01_A11-1124987480,NN/Glia,NN,Oligodendrocyte
AAACAGCCAGGCTACT-L8XR_210812_01_A11-1124987480,NN/Glia,Glia,Microglia
AAACATGCAATCTCTC-L8XR_210812_01_A11-1124987480,Neuron,IN_Str_CGE,IN_Str_VIP
AAACCAACAGTAGCCT-L8XR_210812_01_A11-1124987480,Neuron,IN_Str_MGE,IN_Str_SST
...,...,...,...
TTTGTGGCAGCAAGTG-L8XR_210909_02_C09-1132816395,MSN,D2-MSN,D2-Striosome
TTTGTGTTCGAAGCGG-L8XR_210909_02_C09-1132816395,MSN,D1-MSN,D1-Matrix
TTTGTGTTCGCACAAT-L8XR_210909_02_C09-1132816395,NN/Glia,NN,Oligodendrocyte


In [11]:
exp_data = ExpressionDataset(
    expression_data=expression,
    annotation_data=annotation,
    expression_type='raw',
)
exp_data.expression_data.shape

(81525, 23534)

In [12]:
#  Shorten gene list to speed up PROPOSE 
seurat = SeuratMethod()
hv_selection = seurat.select_gene_panel(size=10000, data=exp_data, flavor='seurat_v3')
hv_data = hv_selection.expression_dataset()
hv_data.expression_data.shape

(81525, 10000)

In [14]:
propose = PROPOSEMethod()
selection = propose.select_gene_panel(size=100, data=hv_data, use_classes=True, annotation_column='supertype_label')

81525 total examples, 65220 training examples, 8153 validation examples, 8152 test examples
using CrossEntropyLoss, starting with lam = 0.0001


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.000100 yielded 4830 genes
Warm starting model for next iteration
next attempt is lam = 0.001000


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = 0.001000 yielded 144 genes
Reinitializing model for next iteration
next attempt is lam = 0.000310


...

Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = nan yielded 9962 genes
Warm starting model for next iteration
next attempt is lam = nan


Training epochs:   0%|          | 0/500 [00:00<?, ?it/s]

lam = nan yielded 9961 genes
Warm starting model for next iteration
next attempt is lam = nan


ValueError: reached maximum number of trials without selecting the desired number of genes! The results may have large variance due to small dataset size, or the initial lam value may be bad

In [ ]:
list(selection.gene_panel)

In [ ]:
# Report on what happened in the final selection process
selection.report()

In [ ]:
# Evaluation of final gene panel
#   number of DE genes per cluster
#   overall expression levels too high/low?
#   includes genes used for sanity checking
#   excludes genes known not to work (per-ST-method)
#   classification accuracy, confusion matrix
#   manifold preservation
evaluate_gene_panel(selection)